In progress ** 

Model based reinforcement learning in a gridworld environment using a model-based Dyna-Q agent. 

Based on code by Kirth Sathiyakumar and Annik Carson

---



In [2]:
# import relevant packages
import numpy as np
import matplotlib.pyplot as plt
from pylab import *

# we have offloaded our classes defined previously to py files which we need to import here to use in this notebook 
from google.colab import drive
drive.mount('/content/drive')
import sys

sys.path.insert(0,'/content/drive/My Drive/Colab Notebooks/CoOp Project/') ## will have to change this path! 

# import premade Gridworlds from Week2 Tutorial
from pyfiles.GridworldEnvs import BoundedGridworld, EdgewrapGridworld, CliffWorld

# import utility functions
from pyfiles.Utils import running_mean as rm
from pyfiles.Utils import oneD2twoD

Mounted at /content/drive


In [3]:
class DynaAgent:
    
    def __init__(self, exp_rate=0.1, lr=0.1, n_steps=50, episodes=1):
        
        self.maze = env
        self.state = env.state1D
        self.actions = env.action_list
        self.state_actions = []  # state & action track
        self.exp_rate = 0.3
        self.lr = 0.1
        
        self.steps = n_steps
        self.episodes = episodes  # number of episodes going to play
        self.steps_per_episode = []
        self.rwd_tracker = []
        self.Q_values = {}
        
        # model function
        self.model = {}
        for row in range(env.r):
            for col in range(env.c):
                self.Q_values[(row, col)] = {}
                for a in self.actions:
                    self.Q_values[(row, col)][a] = 0
        
    def chooseAction (self):
        # Epsilon-greedy agent policy
        state = env.state1D
        state2d = env.state2D
        actions_allowed = env.get_actions()
        
        if random.uniform(0, 1) <= self.exp_rate:
            # explore
            choose = np.asarray([*env.action_dict.values()])[actions_allowed]
            choose_1 = np.asarray(env.action_list)[actions_allowed]
            choice = np.random.choice(choose)
            return (choice)

        else:
            # exploit on allowed actions
            actions_allowed = np.asarray([*env.action_dict.values()])[actions_allowed]
            Q_s = self.Q_values[state2d]
            actions_greedy_1 = actions_allowed[np.flatnonzero(Q_s == np.max(Q_s))]
            choice_number = np.random.choice(actions_greedy_1)
            return (choice_number)
    
    def reset(self):
        self.maze = env
        self.state = env.state1D
        self.state_actions = []
        
    def play_dummy(self):
        self.reset()
        rwd_tracker = []
        
        for ep in range(self.episodes):
            #reset env to baseline 
            env.resetEnvironment()
            
            #reset cumulative reward to 0 
            cum_rwd = 0
            
            #step through          
            for step in range (self.steps):
                #get state
                state2D = env.state2D
                
                #select action
                action = self.chooseAction()
                action_str = self.actions[action]
                
                #move
                nxtState, reward, done = env.move(action)
                #print(reward)
                
                #update q dictionary 
                self.Q_values[state2D][action_str] += self.lr*(reward + np.max(list(self.Q_values[nxtState].values())) - self.Q_values[state2D][action_str])
                
                #add reward to counter 
                cum_rwd += reward 
                
            rwd_tracker.append(cum_rwd)
            
            if ep % 50 == 0:
                print(f"Episode:{ep} Rewards:{cum_rwd}")
    
    def play(self, n_timesteps,print_frequency=25): 
        #reseting for fresh run 
        self.reset()
        self.rwd_tracker = []
        
        for ep in range(self.episodes):    
            env.resetEnvironment()# starting state
            cum_rwd = 0 #reset cumulative reward to 0 
            state1D = env.state1D
            state2D = env.state2D
            
            for time_steps in range(n_timesteps):
                #choose action and take a step in the env 
                action = self.chooseAction()
                action_str = self.actions[action]
                nxtState, reward, done = env.move(action)
                
                # update Q-value
                self.Q_values[state2D][action_str] += self.lr*(reward + np.max(list(self.Q_values[nxtState].values())) - self.Q_values[state2D][action_str])
                
                #track reward 
                cum_rwd += reward
                
                # add transition to model
                if state2D not in self.model.keys():
                    self.model[state2D] = {}
                self.model[state2D][action_str] = (reward, nxtState)
                self.state = nxtState
            
            #add total episode reward     
            self.rwd_tracker.append(cum_rwd)

            # update Q-values based on planning from memory 
            for step_number in range(self.steps):
                # randomly choose an state
                rand_idx = np.random.choice(range(len(self.model.keys()))) #randomly sample one state from the model dict
                _state = list(self.model)[rand_idx] 

                # randomly choose an action
                rand_idx = np.random.choice(range(len(self.model[_state].keys()))) #randomly sample one action from the state dict of the model 
                _action_str = list(self.model[_state])[rand_idx]
                _reward, _nxtState = self.model[_state][_action_str]
                #update the q value based on this random sample from memory
                self.Q_values[_state][_action_str] += self.lr*(_reward + np.max(list(self.Q_values[_nxtState].values())) - self.Q_values[_state][_action_str])       


            if ep % print_frequency == 0:
                print(f"Episode:{ep} Rewards:{sum(self.rwd_tracker[-print_frequency:])}")
    
        return self.rwd_tracker
    